Explicando predicciones utilizando LIME
=======================================

Introducción
------------

AllenNLP es un framework general de aprendizaje profundo para NLP, establecido por el mundialmente famoso Allen Institute for AI Lab. Contiene modelos de referencia de última generación que se ejecutan sobre el `PyTorch`. AllenNLP es una librería que ademas busca implementar abstracciones que permitan el rápido desarrollo de modelos y reutilización de componentes al despegarse de detalles de implementación de cada modelo.

En este ejemplo, veremos como utilizar esta librería para generar salency maps que nos permitan interpretar las predicciones de nuestros modelos basados en `transformers`.

### Para ejecutar este notebook

Para ejecutar este notebook, instale las siguientes librerias:

In [1]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv \
    --quiet --no-clobber --directory-prefix ./Datasets/mascorpus/

!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/TextDataset.py \
    --quiet --no-clobber --directory-prefix ./Utils/
    
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/docs/nlp/neural/allennlp_interpret.txt \
    --quiet --no-clobber
!pip install -r allennlp_interpret.txt --quiet

     |████████████████████████████████| 3.1 MB 5.5 MB/s 
     |████████████████████████████████| 738 kB 43.4 MB/s 
ERROR: Could not find a version that satisfies the requirement allennlp-interpret (from versions: none)
ERROR: No matching distribution found for allennlp-interpret


Si ejecuta en Google Colab, adicionalmente deberá:

In [1]:
!pip install transformers
!pip install eli5

     |████████████████████████████████| 3.1 MB 5.1 MB/s 
     |████████████████████████████████| 59 kB 6.4 MB/s 
     |████████████████████████████████| 895 kB 58.2 MB/s 
     |████████████████████████████████| 596 kB 63.7 MB/s 
     |████████████████████████████████| 3.3 MB 25.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 106 kB 5.1 MB/s 


Descargaremos un modelo previamente entrenando el el problema de clasificación de Tweets:

In [2]:
!wget https://santiagxf.blob.core.windows.net/public/models/tweet_classification_bert.zip --no-clobber --quiet
!unzip -qq tweet_classification_bert.zip

In [1]:
import warnings
warnings.filterwarnings('ignore')

Cargamos el set de datos

In [82]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "tweet_classification_bert"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [92]:
from typing import List

def predict_proba(text: str) -> List[float]:
  inputs = tokenizer(text, padding=True, truncation=True, max_length=20, return_tensors='pt')
  predictions = model(**inputs)
  smx = torch.nn.Softmax(dim = 1)(predictions.logits)
  #odds = predictions.logits.exp()
  #probs = odds/(odds.sum(axis=1))

  
  #smx = probs.detach().numpy()
  #return np.sum(smx, )
  #smx[smx.argmax(axis=1)]+=1-smx.sum(axis=1)
  #return smx
  return smx.detach().numpy()

In [93]:
import torch

In [94]:
predict_proba(["la casa estaba si vacia claro oppp", "la casa se salio de control"])

array([[0.10890099, 0.02119908, 0.04010604, 0.04518595, 0.02475744,
        0.74086446, 0.01898602],
       [0.17640594, 0.02150902, 0.08571446, 0.10166235, 0.03061507,
        0.54807955, 0.03601361]], dtype=float32)

In [40]:
target_names = ['ALIMENTACION', 'AUTOMOCION', 'BANCA', 'BEBIDAS', 'DEPORTES', 'RETAIL', 'TELCO']

In [97]:
import eli5
from eli5.lime import TextExplainer

te = TextExplainer(random_state=42, n_samples=500).fit("Nos estafaron en mercadona. No vuelvo a comprar alli jamas", predict_proba)

In [98]:
te.show_prediction(target_names=target_names)

In [ ]:
import pandas as pd

tweets = pd.read_csv('Datasets/mascorpus/tweets_marketing.csv')

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tweets['TEXTO'], tweets['SECTOR'], 
                                                    test_size=0.33, 
                                                    stratify=tweets['SECTOR'])

### Verificando el hardware disponible

In [ ]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

print("Este notebook se está ejecutando en", device)

Este notebook se está ejecutando en cuda


Sobre los salency maps
----------------------

TODO

Cargamos nuestro modelo en allennlp
-----------------------------------

Verifiquemos la performance de nuestro modelo

In [ ]:
model_name = "tweet_classification"

### Creando un objeto Model

Importamos algunos elementos que necesitaremos

In [ ]:
from typing import Dict, Iterable, List

from allennlp.common import Params
from allennlp.data import DatasetReader, Instance, Batch
from allennlp.data.fields import Field, LabelField, TextField
from allennlp.data.token_indexers import TokenIndexer
from allennlp.data.tokenizers import Tokenizer
from allennlp.data.vocabulary import PreTrainedTokenizer, Vocabulary
from allennlp.models import BasicClassifier, Model
from allennlp.modules.token_embedders import PretrainedTransformerEmbedder
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.data.tokenizers.pretrained_transformer_tokenizer import PretrainedTransformerTokenizer
from allennlp.data.token_indexers.pretrained_transformer_indexer import PretrainedTransformerIndexer
from allennlp.modules.seq2vec_encoders.bert_pooler import BertPooler

In [ ]:
transformer_vocab = Vocabulary.from_pretrained_transformer(model_name)
transformer_tokenizer = PretrainedTransformerTokenizer(model_name)
transformer_encoder = BertPooler(model_name)

token_indexer = PretrainedTransformerIndexer(model_name)

In [ ]:
params = Params(
    {
     "token_embedders": {
        "tokens": {
          "type": "pretrained_transformer",
          "model_name": model_name,
        }
      }
    }
)

token_embedder = BasicTextFieldEmbedder.from_params(vocab=vocab, params=params)

In [ ]:
transformer_model = BasicClassifier(vocab=transformer_vocab, text_field_embedder=token_embedder, seq2vec_encoder=transformer_encoder, dropout=0.1, num_labels=7)

### Creamos un DatasetReader

In [ ]:
from allennlp.data import DatasetReader

In [ ]:
class ClassificationTransformerReader(DatasetReader):
    def __init__(
        self,
        tokenizer: Tokenizer,
        token_indexer: TokenIndexer,
        max_tokens: int,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.tokenizer = tokenizer
        self.token_indexers: Dict[str, TokenIndexer] = { "tokens": token_indexer }
        self.max_tokens = max_tokens
        self.vocab = vocab

    def text_to_instance(self, text: str, label: str = None) -> Instance:
        tokens = self.tokenizer.tokenize(text)
        if self.max_tokens:
            tokens = tokens[: self.max_tokens]
        
        inputs = TextField(tokens, self.token_indexers)
        fields: Dict[str, Field] = { "tokens": inputs }
            
        if label:
            fields["label"] = LabelField(label)
            
        return Instance(fields)

Instanciamos el `DatasetReader`

In [ ]:
dataset_reader = ClassificationTransformerReader(tokenizer=tokenizer, token_indexer=indexer, max_tokens=400)

### Provemos que nuestro modelo funciona

In [ ]:
instance = dataset_reader.text_to_instance(sample_text)

In [ ]:
dataset = Batch([instance])
dataset.index_instances(vocab)

In [ ]:
from allennlp.nn import util
model_input = util.move_to_device(dataset.as_tensor_dict(), model._get_prediction_device())

In [ ]:
outputs = model.make_output_human_readable(model(**model_input))

In [ ]:
outputs['probs'].argmax()

tensor(4)

Interpretando nuestras predicciones
-----------------------------------

In [ ]:
from allennlp.interpret.saliency_interpreters import SimpleGradient
from allennlp.predictors import Predictor, TextClassifierPredictor

In [ ]:
predictor = TextClassifierPredictor(transformer_model, dataset_reader)

In [ ]:
interpreter = SimpleGradient(predictor)

Busquemos un tweet para interpretar:

In [ ]:
sample_text_idx = 2071
sample_text = tweets['TEXTO'][sample_text_idx]
sample_label = tweets['SECTOR'][sample_text_idx]

print("Texto:", sample_text, "\Sector:", sample_label)

Samples: BBVA remolca el crecimiento de la banca española pese a los obstáculos https://t.co/wF9tOqxB5D 
Label: BANCA


In [ ]:
train_dataset.label_map

{'ALIMENTACION': 0,
 'AUTOMOCION': 1,
 'BANCA': 2,
 'BEBIDAS': 3,
 'DEPORTES': 4,
 'RETAIL': 5,
 'TELCO': 6}

Calculemos los gradientes para cada token:

In [ ]:
inputs = {"sentence": sample_text }

In [ ]:
interpretation = interpreter.saliency_interpret_from_json(inputs)
grads = interpretation['instance_1']['grad_input_1']

Podemos graficar los resultados utilizando un mapa de calor marcando con colores más intensos aquellos tokens que tienen mayor impacto en las predicciones:

In [ ]:
import math
from IPython.display import HTML

html = ""
for idx, token in enumerate(tokenizer.tokenize(inputs['sentence'])):
    html += "<span style='background-color:rgba(255,0,0,{})'>{} </span>".format(grads[idx],token)
    
HTML(html)